In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import butter, filtfilt
import pandas as pd

# List of Specific TXT Files to Process
file_paths = [
    r"C:\Data\RawData\Base_Isolated_Moment_Frame_with_Impact_on_Concrete_Moat_Wall\filtered_fpi46gm111s3.txt",
    r"C:\Data\RawData\Base_Isolated_Moment_Frame_with_Impact_on_Concrete_Moat_Wall\filtered_fpi46gm171s2.txt",
    r"C:\Data\RawData\Base_Isolated_Moment_Frame_with_Impact_on_Concrete_Moat_Wall\filtered_fpi46gm171s3.txt",
    r"C:\Data\RawData\Base_Isolated_Moment_Frame_with_Impact_on_Concrete_Moat_Wall\filtered_fpi62gm171s1.txt",
    r"C:\Data\RawData\Base_Isolated_Moment_Frame_with_Impact_on_Concrete_Moat_Wall\filtered_fpi62gm171s2.txt",
    r"C:\Data\RawData\Base_Isolated_Moment_Frame_with_Impact_on_Concrete_Moat_Wall\filtered_fpi62gm171s3.txt",
    r"C:\Data\RawData\Base_Isolated_Moment_Frame_with_Impact_on_Concrete_Moat_Wall\filtered_fpi64gm171s2a.txt",
    r"C:\Data\RawData\Base_Isolated_Moment_Frame_with_Impact_on_Concrete_Moat_Wall\filtered_fpi64gm171s3a.txt",
    r"C:\Data\RawData\Base_Isolated_Moment_Frame_with_Impact_on_Concrete_Moat_Wall\filtered_fpi46gm71s3.txt",
    r"C:\Data\RawData\Base_Isolated_Moment_Frame_with_Impact_on_Steel_Moat_Wall\filtered_fp4rgm171s2.txt",
    r"C:\Data\RawData\Base_Isolated_Moment_Frame_with_Impact_on_Steel_Moat_Wall\filtered_fp4rgm171s3.txt",
    r"C:\Data\RawData\Base_Isolated_Moment_Frame_with_Impact_on_Steel_Moat_Wall\filtered_fp4rngm111s3.txt",
    r"C:\Data\RawData\Base_Isolated_Moment_Frame_with_Impact_on_Steel_Moat_Wall\filtered_fp4rngm171s3.txt",
    r"C:\Data\RawData\Base_Isolated_Moment_Frame_with_Impact_on_Steel_Moat_Wall\filtered_fp5rgm171s3.txt",
    r"C:\Data\RawData\Base_Isolated_Moment_Frame_with_Impact_on_Steel_Moat_Wall\filtered_fp6rgm171s3.txt",
    r"C:\Data\RawData\Base_Isolated_Moment_Frame_with_Impact_on_Steel_Moat_Wall\filtered_fp6rngm171s3.txt",
    r"C:\Data\RawData\Base_Isolated_Moment_Frame_with_Impact_on_Steel_Moat_Wall\filtered_fp4rgm111s3.txt",
    r"C:\Data\RawData\Base_Isolated_Moment_Frame_with_Impact_on_Steel_Moat_Wall\filtered_fp4rngm71s3.txt",
    r"C:\Data\RawData\Base_Isolated_Moment_Frame_with_Impact_on_Steel_Moat_Wall\filtered_fp4rgm71s3.txt",
    r"C:\Data\RawData\Base_Isolated_Moment_Frame_without_Impact\filtered_fpgm111s3.txt",
    r"C:\Data\RawData\Base_Isolated_Moment_Frame_without_Impact\filtered_fpgm152s1.txt",
    r"C:\Data\RawData\Base_Isolated_Moment_Frame_without_Impact\filtered_fpgm152s2.txt",
    r"C:\Data\RawData\Base_Isolated_Moment_Frame_without_Impact\filtered_fpgm152s3.txt",
    r"C:\Data\RawData\Base_Isolated_Moment_Frame_without_Impact\filtered_fpgm162s1.txt",
    r"C:\Data\RawData\Base_Isolated_Moment_Frame_without_Impact\filtered_fpgm162s2.txt",
    r"C:\Data\RawData\Base_Isolated_Moment_Frame_without_Impact\filtered_fpgm162s3.txt",
    r"C:\Data\RawData\Base_Isolated_Moment_Frame_without_Impact\filtered_fpgm171s1.txt",
    r"C:\Data\RawData\Base_Isolated_Moment_Frame_without_Impact\filtered_fpgm171s2.txt",
    r"C:\Data\RawData\Base_Isolated_Moment_Frame_without_Impact\filtered_fpgm171s3.txt",
    r"C:\Data\RawData\Base_Isolated_Moment_Frame_without_Impact\filtered_fpgm172s1.txt",
    r"C:\Data\RawData\Base_Isolated_Moment_Frame_without_Impact\filtered_fpgm172s2.txt",
    r"C:\Data\RawData\Base_Isolated_Moment_Frame_without_Impact\filtered_fpgm172s3.txt",
    r"C:\Data\RawData\Base_Isolated_Moment_Frame_without_Impact\filtered_fpgm111s1.txt",
    r"C:\Data\RawData\Base_Isolated_Moment_Frame_without_Impact\filtered_fpgm111s2.txt",
    r"C:\Data\RawData\Fixed_Base_Moment_Frame\filtered_fbgm162s2.txt",
    r"C:\Data\RawData\Fixed_Base_Moment_Frame\filtered_fbgm162s3.txt",
    r"C:\Data\RawData\Fixed_Base_Moment_Frame\filtered_fbgm162s4.txt",
    r"C:\Data\RawData\Fixed_Base_Moment_Frame\filtered_fbgm162s5.txt",
    r"C:\Data\RawData\Fixed_Base_Moment_Frame\filtered_fbgm152s1.txt",
    r"C:\Data\RawData\Fixed_Base_Moment_Frame\filtered_fbgm152s2.txt",
    r"C:\Data\RawData\Fixed_Base_Moment_Frame\filtered_fbgm152s3.txt",
    r"C:\Data\RawData\Fixed_Base_Moment_Frame\filtered_fbgm152s4.txt",
    r"C:\Data\RawData\Fixed_Base_Moment_Frame\filtered_fbgm162s1.txt"    
]

# Option to Turn On/Off Plotting
enable_plotting = True  # Set to False to disable plotting

for file_path in file_paths:
    print(f"\nProcessing file: {file_path}")

    # Load Experimental Data
    data = pd.read_csv(file_path, sep=r'\s+', header=0)  # Read with header row (parameter names)

    # File Path for Saving Denoised Data
    output_file = file_path.replace(".txt", "_denoised.txt")

    # Extract Time and Recorded Parameters
    time = data.iloc[1:, 0].astype(float).values  # Time column (skipping units row)
    parameters = data.columns[1:]  # Names of recorded parameters
    units = data.iloc[0, 1:]  # Units of each parameter

    # Extract Recorded Values (excluding header and unit rows)
    signals = data.iloc[1:, 1:].astype(float).values  # Recorded values for each parameter

    # Automatically Calculate Sampling Frequency (fs)
    dt = np.diff(time).mean()  # Average time step
    fs = 1 / dt  # Sampling frequency
    nyquist = 0.5 * fs  # Nyquist frequency

    # Define Filter Settings for Each Parameter
    filter_settings = {
        'aextx': {'type': 'low', 'cutoff': [min(120, nyquist - 0.01)]},
        's2': {'type': 'low', 'cutoff': [min(25, nyquist - 0.01)]},
        's3': {'type': 'low', 'cutoff': [min(25, nyquist - 0.01)]},
        's4': {'type': 'low', 'cutoff': [min(25, nyquist - 0.01)]},
        's5': {'type': 'low', 'cutoff': [min(25, nyquist - 0.01)]},
        'dWestWall': {'type': 'low', 'cutoff': [min(20, nyquist - 0.01)]},
        'dEastWall': {'type': 'low', 'cutoff': [min(20, nyquist - 0.01)]},
        'Vs2': {'type': 'low', 'cutoff': [min(20, nyquist - 0.01)]},
        'Vs3': {'type': 'low', 'cutoff': [min(20, nyquist - 0.01)]},
        'Vs4': {'type': 'low', 'cutoff': [min(20, nyquist - 0.01)]},
        'Vs5': {'type': 'low', 'cutoff': [min(20, nyquist - 0.01)]},
        'VdWestWall': {'type': 'low', 'cutoff': [min(20, nyquist - 0.01)]},
        'VdEastWall': {'type': 'low', 'cutoff': [min(20, nyquist - 0.01)]},
        'ACCs2': {'type': 'low', 'cutoff': [min(15, nyquist - 0.01)]},
        'ACCs3': {'type': 'low', 'cutoff': [min(15, nyquist - 0.01)]},
        'ACCs4': {'type': 'low', 'cutoff': [min(15, nyquist - 0.01)]},
        'ACCs5': {'type': 'low', 'cutoff': [min(15, nyquist - 0.01)]},
        'ACCdWestWall': {'type': 'low', 'cutoff': [min(35, nyquist - 0.01)]},
        'ACCdEastWall': {'type': 'low', 'cutoff': [min(35, nyquist - 0.01)]},
        'FCEast': {'type': 'low', 'cutoff': [min(80, nyquist - 0.01)]},
        'FCWest': {'type': 'low', 'cutoff': [min(80, nyquist - 0.01)]},
    }

    # Define Butterworth Filter Function
    def butterworth_filter(signal, fs, cutoff, filter_type, order=4):
        nyquist = 0.5 * fs
        normal_cutoff = np.clip(np.array(cutoff) / nyquist, 0.0001, 0.9999)
        b, a = butter(order, normal_cutoff, btype=filter_type, analog=False)
        return filtfilt(b, a, signal)

    # Apply Filter to Each Parameter
    filtered_signals = []
    for i, param in enumerate(parameters):
        settings = filter_settings.get(param, {'type': 'low', 'cutoff': [min(0.2 * nyquist, nyquist - 0.01)]})
        filtered_signals.append(butterworth_filter(signals[:, i], fs, settings['cutoff'], settings['type']))

    # Plotting Each Parameter Separately (Only if Enabled)
    if enable_plotting:
        for i, param in enumerate(parameters):
            plt.figure(figsize=(12, 6))
            plt.plot(time, signals[:, i], label=f'Original {param} ({units[i]})', alpha=0.6)
            plt.plot(time, filtered_signals[i], label=f'Filtered {param}', linestyle='--')
            plt.title(f'{param} - Filtered (Low-Pass)')
            plt.xlabel('Time (s)')
            plt.ylabel(f'{param} ({units[i]})')
            plt.legend(loc='upper right')
            plt.grid(True, linestyle='--', alpha=0.6)
            plt.show()

    # Save Denoised Data to TXT File
    denoised_data = np.column_stack([time] + filtered_signals)
    column_names = ['Time'] + list(parameters)
    denoised_df = pd.DataFrame(denoised_data, columns=column_names)
    denoised_df.to_csv(output_file, sep='\t', index=False)

    print(f"\nDenoised data saved to: {output_file}")
